In [3]:
import sys
sys.path.append( '/anaconda/envs/py37_pytorch/lib/python3.7')
sys.path.append('/anaconda/envs/py37_pytorch/lib/python3.7/site-packages')
sys.path.append('/home/v-zeyyan/.local/lib/python3.6/site-packages')

In [4]:
%load_ext autoreload
%autoreload 2
import os

import yaml
import glob
import logging
import numpy as np
from importlib import reload  # Not needed in Python 2

import torch
from torch import nn
import torch.nn.functional as F
from torchvision.models import ResNet
from torch.utils.data import DataLoader,Dataset
import seaborn as sns


from dataset.ASdataset import AS_Data
from dataset.ASdataset_obs_train_input import AS_Data_obs

device = torch.device("cuda:1"  if torch.cuda.is_available() else "cpu")
reload(logging)
logging.basicConfig(level=logging.INFO,#控制台打印的日志级别
                    filename='log/step2-reverse-model-logging.txt',
                    filemode='a',##模式，有w和a，w就是写模式，每次都会重新写日志，覆盖之前的日志
                    #a是追加模式，默认如果不写的话，就是追加模式
                    format=
                    '%(asctime)s : %(message)s',
                    )

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline



# class Focal_loss_regression(nn.Module):
#     def __init__(self,max_update=10,_lambda=2,):
#         super(Focal_loss_regression,self).__init__()
#         self._lambda = _lambda
#         max_update = np.power(1/max_update,1/_lambda)
#         max_update = 1/max_update
#         max_update = 1/(max_update-1)
#         self.max_update = max_update
        
#     def forward(self,pred,target):
#         diff_abs = torch.abs(pred-target)
#         diff_max = (1+self.max_update)*torch.max(diff_abs)
# #         diff_max.detach()
#         rate = torch.pow((1-1/diff_max*diff_abs)**self._lambda,-1)
#         diff_abs = rate*diff_abs
        
# #         return diff_abs
#         return torch.mean(diff_abs)

In [5]:
with open('config/cfg.yaml','r') as f:
    cfg = yaml.load(f)

cfg = {**cfg['step2'],**cfg['share_cfg']}

T = cfg['T']
pollution = cfg['pollution']
batch_size = cfg['batch_size']
EM_idx = np.array(cfg['EM_idx'])

print('train data is loading ')
Data = AS_Data(cfg['data_path'],left = cfg['train']['left'],right = cfg['train']['right'],window = T,EM_idx = EM_idx,pollution = pollution)
trainloader = DataLoader(Data,batch_size=batch_size,shuffle=True)
print(len(Data))

# print('test data is loading ')
# test_Data = AS_Data(cfg['data_path'],left = cfg['test']['left'],right = cfg['test']['right'],window = T,pollution = pollution)
# testloader = DataLoader(test_Data,batch_size=batch_size,shuffle=True)
# print(len(test_Data))

<ipython-input-5-6f6bd9445ea2>:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(f)


train data is loading 
/AS_data/Conc_npy/TOTAL_2015_01_NO2_SO2_O3_PM25_PM10_CO__744_6_182_232.npy   is loading
/AS_data/Conc_npy/TOTAL_2015_04_NO2_SO2_O3_PM25_PM10_CO__720_6_182_232.npy   is loading
/AS_data/Conc_npy/TOTAL_2015_07_NO2_SO2_O3_PM25_PM10_CO__744_6_182_232.npy   is loading
/AS_data/Conc_npy/TOTAL_2015_10_NO2_SO2_O3_PM25_PM10_CO__744_6_182_232.npy   is loading
(744, 4, 182, 232)
/AS_data/zeyuan_folder/concat_data/rest_EM_2015_01_744_07_182_232.npy   is loading
/AS_data/zeyuan_folder/concat_data/rest_EM_2015_04_720_07_182_232.npy   is loading
/AS_data/zeyuan_folder/concat_data/rest_EM_2015_07_744_07_182_232.npy   is loading
/AS_data/zeyuan_folder/concat_data/rest_EM_2015_10_744_07_182_232.npy   is loading
(744, 7, 182, 232)
/AS_data/METCRO2D_npy/METCRO2D_2015_01_744_34_182_232.npy   is loading
/AS_data/METCRO2D_npy/METCRO2D_2015_04_720_34_182_232.npy   is loading
/AS_data/METCRO2D_npy/METCRO2D_2015_07_744_34_182_232.npy   is loading
/AS_data/METCRO2D_npy/METCRO2D_2015_10_744

In [6]:
from model.res_model_LSTM import res8
from model.unet_model_LSTM import UNet
from model.layers import Tensor_Parameter

name = cfg['name']

# name = 'res_2layer_correctdata'
# test_model.load_state_dict(torch.load('model_save/res_2layer_9_epoch.t'))
test_model = UNet(cfg['meteorological_dim']+cfg['emission_dim'],cfg['grid_dim'],T=T,bilinear=False,pre_dim = len(pollution)) #+80
# test_model = res8(cfg['meteorological_dim']+cfg['emission_dim'],cfg['grid_dim'],T=T,pre_dim = len(pollution))
# t2p = Tensor_Parameter()


test_model.to(device)
# t2p.to(device)
# criterion = torch.nn.L1Loss()
# optimizer = torch.optim.SGD(t2p.parameters(),lr=1)
# optimizer = torch.optim.Adam(t2p.parameters(),lr=1e-1)
test_model.load_state_dict(torch.load('model_save/model_F_35_epoch.t'))
# test_model.load_state_dict(torch.load('model_save/o3_best_unet2_1month_65_epoch.t'))


<All keys matched successfully>

In [7]:
from model.unet_model_LSTM import ReverseModel
reverse_model = ReverseModel(cfg['meteorological_dim'],cfg['grid_dim'],pre_dim = len(pollution),T=T,em_dim=cfg['emission_dim'])
reverse_model.to(device)
criterion = torch.nn.L1Loss()
reverse_optimizer = torch.optim.Adam(reverse_model.parameters(),lr=1e-2)

In [8]:
def print_logging(string):
    print(string)
    logging.info(string)


def score(testloader):
    eval_loss = []
    eval_direct_loss = []
    
    print_logging('********Evaluating*********')
    for idx,i in enumerate(testloader):
        with torch.no_grad():
            indexes,input1_24,grid,yt_1,label_24,next_label24_47, next_metro24_47 = [j.to(device) for j in i]
            em1_24 = input1_24[:,:,:cfg['emission_dim'],:,:]
            metro1_24 = input1_24[:,:,cfg['emission_dim']:,:,:]

            em_24_pred = reverse_model(next_metro24_47,grid,next_label24_47)
            new_em = torch.cat([em1_24[:,:-1,:,:,:],em_24_pred],dim=1)
            new_em = torch.cat([new_em,metro1_24],dim=2)

            label_pred_24 = test_model(new_em,grid,yt_1) #输入yt_1但是模型中没用

            direct_loss = criterion(em_24_pred.detach(),em1_24[:,-1:,:,:,:])
            
            cur_loss = []
            for j in range(label_24.shape[1]):
                loss = criterion(label_pred_24[:,j],label_24[:,j])
                cur_loss.append(loss.cpu().data)
            eval_loss.append(cur_loss)
            
            eval_direct_loss.append(direct_loss.cpu().data)
        
    print_logging(f'------------Evaluating: Direct Loss: {np.mean(np.array(eval_direct_loss))};')
    print_logging(' F Loss for pm25 o3 no2 so2 ,{}, {}, {},{}'.format(*np.mean(np.array(eval_loss),axis = 0)))
    
    
    return np.mean(np.array(eval_direct_loss)),*np.mean(np.array(eval_loss),axis = 0)

In [ ]:
score(trainloader)
print_logging('Train the reverse model')


for epoch in range(1,100):
    print_logging('-----------{}-----------'.format(epoch))
    ls = []
    direct_ls = []
    reverse_model.train()
    test_model.train()
    count = 0
    for idx,i in enumerate(trainloader):
        indexes,input1_24,grid,yt_1,label_24,next_label24_47, next_metro24_47 = [j.to(device) for j in i]
        em1_24 = input1_24[:,:,:cfg['emission_dim'],:,:]
        metro1_24 = input1_24[:,:,cfg['emission_dim']:,:,:]
        
        reverse_optimizer.zero_grad()
        
        em_24_pred = reverse_model(next_metro24_47,grid,next_label24_47)
        new_em = torch.cat([em1_24[:,:-1,:,:,:],em_24_pred],dim=1)
        
        new_input = torch.cat([new_em,metro1_24],dim=2)
        
        label_pred_24 = test_model(new_input,grid,yt_1) #输入yt_1但是模型中没用
        loss = criterion(label_pred_24,label_24)
        loss.backward()
        reverse_optimizer.step()
        direct_loss = criterion(em_24_pred.detach(),em1_24[:,-1:,:,:,:])
        count += 1
        if count%100 == 0:print(f'         Direct Loss: {np.mean(np.array(direct_ls))}; F Loss: {np.mean(np.array(ls))}')

        ls.append(loss.cpu().data)
        direct_ls.append(direct_loss.cpu().data)
        
        
        ### update input data
        Data.update(indexes,new_em)
        
    print_logging(f'Average Training Loss: Direct Loss: {np.mean(np.array(direct_ls))}; F Loss: {np.mean(np.array(ls))}')
    
    if epoch%5 == 0:
        torch.save(reverse_model.cpu().state_dict(),'model_save/model_G_{}_epoch.t'.format(epoch))
        reverse_model.to(device)
    
    score(trainloader)


-----------1-----------
         Direct Loss: 0.45309576392173767; F Loss: 1.5053843259811401
         Direct Loss: 0.4092447757720947; F Loss: 1.5169936418533325
         Direct Loss: 0.3957236409187317; F Loss: 1.4778181314468384
         Direct Loss: 0.39836347103118896; F Loss: 1.492730736732483
         Direct Loss: 0.39708754420280457; F Loss: 1.4856045246124268
         Direct Loss: 0.4001055359840393; F Loss: 1.4908708333969116
         Direct Loss: 0.4062238931655884; F Loss: 1.4854233264923096
         Direct Loss: 0.4005134105682373; F Loss: 1.4770941734313965
         Direct Loss: 0.3992975354194641; F Loss: 1.4892369508743286
         Direct Loss: 0.3994910418987274; F Loss: 1.484359860420227
         Direct Loss: 0.3972516655921936; F Loss: 1.4854202270507812
         Direct Loss: 0.39556261897087097; F Loss: 1.4837602376937866
         Direct Loss: 0.39538225531578064; F Loss: 1.484139323234558
         Direct Loss: 0.39653104543685913; F Loss: 1.4850088357925415
       